<b>matplotlib inline</b> : the output of plotting commands is displayed inline within frontends like the Jupyter notebook, directly below the code cell that produced it. The resulting plots will then also be stored in the notebook document

In [1]:
%matplotlib inline

# Import Libraries

In [2]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from matplotlib import pyplot as plt
import numpy as np
from IPython import display
from tensorboardX import SummaryWriter
import torchvision.utils as vutils

# Data Location

In [3]:
DATA_FOLDER = '/home/pushpull/mount/intHdd/dataset/'

In [4]:
# Clubs all the transforms provided to it. So, all the transforms in the (transforms.Compose) are applied to the input one by one

def load_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((.5,), (.5,))
        ])
    out_dir = '{}/'.format(DATA_FOLDER)
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

In [5]:
# Load Dataset

data = load_data()

# Discriminator Network

<pre>




                     [no_data, gan_out_size]

                                ||
                                ||
                                ||
                               \||/
                                \/
                       ____________________
                      |                    |
                      |   LeakyRelu(0.3)   |
            hidden0   |                    |   [no_data, gan_out_size] X [gan_out_size, 512]  ->  [no_data, 512]
                      |    Dropout(0.2)    |
                      |                    |
                      |____________________|

                                ||
                                ||
                          [no_data, 512]
                                ||
                               \||/
                                \/
                       ____________________
                      |                    |
                      |   LeakyRelu(0.3)   |
            hidden1   |                    |   [no_data, 512] X [512, 256]  ->  [no_data, 256]
                      |    Dropout(0.2)    |
                      |                    |
                      |____________________|

                                ||
                                ||
                          [no_data, 256]
                                ||
                               \||/
                                \/
                       ____________________
                      |                    |
                      |   LeakyRelu(0.3)   |
            hidden2   |                    |   [no_data, 256] X [256, 128]  ->  [no_data, 128]
                      |    Dropout(0.2)    |
                      |                    |
                      |____________________|

                                ||
                                ||
                          [no_data, 128]
                                ||
                               \||/
                                \/
                       ____________________
                      |                    |
                      |                    |
            out       |      Sigmoid       |   [no_data, 512] X [128, 1]  ->  [no_data, 1]
                      |                    |
                      |                    |
                      |____________________|

                                ||
                                ||
                                ||
                               \||/
                                \/

                           [no_data, 1]

                     Probability of Data entered
                   sampled from same distribution
                      


</pre>

In [6]:
# Discriminator Network

# torch.nn.module    :    https://pytorch.org/docs/stable/nn.html
#                         Base class for all neural network modules
#                         Your models should also subclass this class


class discriminator(torch.nn.Module):
    def __init__(self, parameter):
        super(discriminator, self).__init__()
        self.y = 1
        self.gan_out = parameter.get("gan_out")
        self.Network()
        
    def Network(self):
            self.hidden0 = nn.Sequential( 
                nn.Linear(self.gan_out, 512),
                nn.LeakyReLU(0.3),
                nn.Dropout(0.2)
                ).cuda()
            self.hidden1 = nn.Sequential(
                nn.Linear(512, 256),
                nn.LeakyReLU(0.3),
                nn.Dropout(0.2)
                ).cuda()
            self.hidden2 = nn.Sequential(
                nn.Linear(256, 128),
                nn.LeakyReLU(0.3),
                nn.Dropout(0.2)
                ).cuda()
            self.out = nn.Sequential(
                torch.nn.Linear(128, self.y),
                torch.nn.Sigmoid()
            ).cuda()
            
    def forward(self, x_):
            x_ = self.hidden0(x_)
            x_ = self.hidden1(x_)
            x_ = self.hidden2(x_)
            x_ = self.out(x_)
            return x_

# Generator Network

<pre>




                       [no_data, gan_in_size]

                                ||
                                ||
                                ||
                               \||/
                                \/
                       ____________________
                      |                    |
                      |                    |
            hidden0   |   LeakyRelu(0.3)   |   [no_data, gan_in_size] X [gan_in_size, 128]  ->  [no_data, 128]
                      |                    |
                      |                    |
                      |____________________|

                                ||
                                ||
                          [no_data, 512]
                                ||
                               \||/
                                \/
                       ____________________
                      |                    |
                      |                    |
            hidden1   |   LeakyRelu(0.3)   |   [no_data, 128] X [128, 256]  ->  [no_data, 256]
                      |                    |
                      |                    |
                      |____________________|

                                ||
                                ||
                          [no_data, 256]
                                ||
                               \||/
                                \/
                       ____________________
                      |                    |
                      |                    |
            hidden2   |   LeakyRelu(0.3)   |   [no_data, 256] X [256, 512]  ->  [no_data, 512]
                      |                    |
                      |                    |
                      |____________________|

                                ||
                                ||
                          [no_data, 128]
                                ||
                               \||/
                                \/
                       ____________________
                      |                    |
                      |                    |
            out       |      Sigmoid       |   [no_data, 512] X [512, gan_out_size]  ->  [no_data, gan_out_size]
                      |                    |
                      |                    |
                      |____________________|

                                ||
                                ||
                                ||
                               \||/
                                \/

                      [no_data, gan_out_size]

                     Probability of Data entered
                   sampled from same distribution
                      


</pre>

In [7]:
# Generator Network


class generator(torch.nn.Module):
    def __init__(self, parameter):
        super(generator, self).__init__()
        self.gan_out = parameter.get("gan_out")
        self.gan_in  = parameter.get("gan_in")
        self.Network()
        
    def Network(self):
            self.hidden0 = nn.Sequential(
                nn.Linear(self.gan_in, 128),
                nn.LeakyReLU(0.3)
            ).cuda()
            self.hidden1 = nn.Sequential(            
                nn.Linear(128, 256),
                nn.LeakyReLU(0.3)
            ).cuda()
            self.hidden2 = nn.Sequential(
                nn.Linear(256, 512),
                nn.LeakyReLU(0.3)
            ).cuda()
        
            self.out = nn.Sequential(
                nn.Linear(512, self.gan_out),
                nn.Tanh()
            ).cuda()
            
    def forward(self, x_):
            x_ = self.hidden0(x_)
            x_ = self.hidden1(x_)
            x_ = self.hidden2(x_)
            x_ = self.out(x_)
            return x_

# Noise Generator

In [8]:
# noise : Sample random numbers from normal distribution

def noise(length, size):
    noise = Variable(torch.randn(length, size)).cuda()
    return noise

In [1]:
# Train discriminator network

def train_discriminator(discriminator, no_data, true_data, fake_data, gan_in, d_optimizer, loss):
#   Manually set gradients of optimizer of discriminator network to zero
    d_optimizer.zero_grad()
    
#   Feed forward in discriminator network using true data
    prob_true  = discriminator(true_data)
    true_out   = Variable(torch.ones(no_data, 1)).cuda()
#   Get error that how well discriminator network classifies true data
    error_true = loss(prob_true, true_out)
#   Back propogate in discriminator network to get gradients to update parameter
    error_true.backward()

#   Feed forward in discriminator network using fake data
    prob_false = discriminator(fake_data)
    false_out = Variable(torch.zeros(no_data, 1)).cuda()
#   Get error that how well discriminator network classifies false data
    error_false = loss(prob_false, false_out)
#   Back propogate in discriminator network to get gradients to update parameter
    error_false.backward()
    
#   Perform back propogation step
    d_optimizer.step()
    
    return (error_true + error_false)
    
    
def train_generator(generator, discriminator, no_data, gen_data, gan_in, g_optimizer, loss):
#   Manually set gradients of optimizer of generator network to zero
    g_optimizer.zero_grad()

#   Probability that generator had sampled data from same distribution as training data
    prob  = discriminator(gen_data)
    true_out   = Variable(torch.ones(no_data, 1)).cuda()
#   Get error that how well generator samples data from same distribution as training data to be discriminated from discriminator
    error = loss(prob, true_out)
    
#   Back propogate in gradient network to get gradients to update parameter
    error.backward()
    
#   Perform back propogation step
    g_optimizer.step()
    
    return error
    
    
def sample(generator, sample_size, in_size, step):
#   Sample input data
    noise_input = noise(sample_size, in_size).cuda()
#   Generate pseudo data
    output      = generator(noise_input).cpu().detach()
    plot_sample(output, sample_size, step)
    
    
def plot_sample(image, sample_size, step):
    image      = image.view(image.size(0), 1, 28, 28)
    grid = vutils.make_grid(image, nrow=int(np.sqrt(sample_size)), normalize=True, scale_each=True)
    fig = plt.figure(figsize=(sample_size, sample_size))
    location = "Step" + str(step) + ".jpg"
    plt.imsave(location, np.moveaxis(grid.numpy(), 0, -1))
    plt.imshow(np.moveaxis(grid.numpy(), 0, -1))
    display.display(plt.gcf())
    plt.close()
    
    
def train(parameter):
    no_epochs = parameter.get("no_epochs")
    gan_in  = parameter.get("gan_in")
    gan_out = parameter.get("gan_out")
    loss = parameter.get("loss")
    x = parameter.get("true_data")
    generator = parameter.get("generator")
    discriminator = parameter.get("discriminator")
    d_optimizer = parameter.get("d_optimizer")
    g_optimizer = parameter.get("g_optimizer")
    sample_size = parameter.get("sample_size")
    no_data = parameter.get("batch_size")
    
    step = 0
    g_error = []
    d_error = []
    n_error = []
    
    for epoch_no in range(no_epochs):
        for i, (data,_) in enumerate(x):
            
            true_data = data.view(batch_size, gan_out)
            true_data = Variable(true_data).cuda()

#           Generate random (fake) data for input in generator network to train discriminator network
            fake_in_data = noise(no_data, gan_in)
#           Detach fake data tensor from graph so not counting it in gradient descent
            fake_data = generator(fake_in_data).detach()
#           Train discriminator network
            disc_error   = train_discriminator(discriminator, no_data, true_data, fake_data, gan_in, d_optimizer, loss)

#           Generate random (fake) data for input in generator network to train generator network
            gen_in_data = noise(no_data, gan_in)
            gen_data = generator(fake_in_data)
#           Train generator network
            gen_error    = train_generator(generator, discriminator, no_data, gen_data, gan_in, g_optimizer, loss)
            
            error        = (gen_error + disc_error)

#           Append errors into list to create graph
            g_error.append(gen_error)
            d_error.append(disc_error)
            n_error.append(error)

#           Generate data after every 10 steps of iteration
            if step%10 == 0:
                display.clear_output(True)
                print("Epoch Number:    ", epoch_no)
                print("Batch Number:    ", i)
            
                print("Generator Error        :    ", gen_error.item())
                print("Discriminator Error    :    ", disc_error.item())

#               Sample data to generate image from given distribution
                sample(generator, sample_size, gan_in, step)

#               Plot error graph
                plt.plot(g_error)
                plt.show()
                plt.plot(d_error)
                plt.show()
                plt.plot(n_error)
                plt.show()
            step += 1

In [10]:
# Set batch_size and input size and output size of gan

batch_size   = 1000
gan_in       = 100
gan_out      = 784

In [11]:
disc_parameter    = {"gan_out":gan_out}

In [12]:
disc = discriminator(disc_parameter)

In [17]:
gen_parameter    = {"gan_out":gan_out,
                    "gan_in":gan_in}

In [18]:
x = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

In [19]:
gen = generator(gen_parameter)

In [20]:
no_epochs = 200
loss = nn.BCELoss()
true_data = x
d_optimizer = optim.Adam(disc.parameters(), lr=0.0002)
g_optimizer = optim.Adam(gen.parameters(), lr=0.0002)
sample_size = 9

train_parameter = {"batch_size":batch_size,
                   "no_epochs":no_epochs,
                   "gan_in":gan_in,
                   "gan_out":gan_out,
                   "loss":loss,
                   "true_data":x,
                   "generator":gen,
                   "discriminator":disc,
                   "d_optimizer":d_optimizer,
                   "g_optimizer":g_optimizer,
                   "sample_size":sample_size}

In [ ]:
train(train_parameter)